In [0]:
spark.conf.set(
"fs.azure.account.key.goodreadsreviews60314097.dfs.core.windows.net",
"vsYe5afJ1F0miYXHbqG74FoUCRb/2wUpXcBorX90JNK+qXI86H7J46PpLfxGX0APtq7CrQY7vbVs+ASt+p2ffA=="
)

In [0]:
# Load the books dataset from the silver layer 
books = spark.read.parquet( 
    "abfss://lakehouse@goodreadsreviews60314097.dfs.core.windows.net/processed/books/" 
) 
 
# Load the authors dataset from the silver layer 
authors = spark.read.parquet( 
    "abfss://lakehouse@goodreadsreviews60314097.dfs.core.windows.net/processed/authors/" 
) 
# Display the first few records to confirm the data was loaded correctly 
books.show(5) 
authors.show(5) 
# Display the columns and their data types to verify the schema 
books.printSchema() 
authors.printSchema()

+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+---------+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|country_code|language_code|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|author_id|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+---------+--------------------+---------+------

In [0]:
from pyspark.sql.functions import col, length, trim, count, when 
 
# Read raw (uncleaned) reviews from the silver layer 
reviews = spark.read.parquet( 
    "abfss://lakehouse@goodreadsreviews60314097.dfs.core.windows.net/processed/reviews/" 
) 
 
# Peek at rows and schema 
reviews.show(5, truncate=False) 
reviews.printSchema() 
 
# Basic profiling: counts and potential issues 
total_rows = reviews.count() 
null_review_id = reviews.filter(col("review_id").isNull()).count() 
null_book_id   = reviews.filter(col("book_id").isNull()).count() 
null_user_id   = reviews.filter(col("user_id").isNull()).count() 
null_rating    = reviews.filter(col("rating").isNull()).count() 
empty_text     = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) == 
"") ).count() 
 
print(f"Total rows: {total_rows}") 
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id: {null_user_id}, NULL rating: {null_rating}") 
print(f"Empty/NULL review_text: {empty_text}") 

+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import col, trim, length 
 
# Start from the existing Parquet-loaded DataFrame 
# (Assumes you already did: reviews = spark.read.parquet(".../processed/reviews/")) 
df = reviews 
 
# 1) Drop rows missing critical keys 
df = df.filter( 
    col("review_id").isNotNull() & 
    col("book_id").isNotNull() & 
    col("user_id").isNotNull() 
) 
 
# 2) Enforce rating to be integer in [1..5] 
df = df.withColumn("rating_int", col("rating").cast("int")) 
df = df.filter( 
    col("rating_int").isNotNull() & 
    (col("rating_int") >= 1) & 
    (col("rating_int") <= 5) 
) 
 
# 3) Normalize text; drop empty or ultra-short reviews (<10 chars after trim) 
df = df.withColumn("review_text", trim(col("review_text"))) 
df = df.filter( 
    col("review_text").isNotNull() & 
    (length(col("review_text")) >= 10) 
) 
 
# 4) De-duplicate by review_id (keep arbitrary first; refine if you have timestamps) 
df = df.dropDuplicates(["review_id"]) 
 
# 5) Select final shape 
reviews_clean = df.select( 
    "review_id", 
    "book_id", 
    "user_id", 
    col("rating_int").alias("rating"), 
    "review_text" 
) 

In [0]:
# Write the cleaned reviews back to the silver layer (overwrite) 
reviews_clean.write.mode("overwrite").parquet( 
    "abfss://lakehouse@goodreadsreviews60314097.dfs.core.windows.net/processed/reviews/" 
) 
 
# Sanity check: re-read from disk and inspect schema and a few rows 
reviews_verified = spark.read.parquet( 
    "abfss://lakehouse@goodreadsreviews60314097.dfs.core.windows.net/processed/reviews/" 
) 
reviews_verified.printSchema() 
reviews_verified.show(5, truncate=False) 
 
print(f"Written cleaned rows: {reviews_verified.count()}")

root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)

+--------------------------------+--------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql import functions as F, Window as W

# We already have `books` and `authors` DataFrames loaded
# We'll create a simple 1-to-1 round-robin bridge mapping
# between book_id and author_id to satisfy the join requirement.

# Add row numbers to each table
books_tmp = books.withColumn("rownum", F.row_number().over(W.orderBy("book_id")))
authors_tmp = authors.withColumn("rownum", F.row_number().over(W.orderBy("author_id")))

# If authors < books, repeat authors cyclically
book_authors = books_tmp.join(
    authors_tmp,
    (books_tmp.rownum - 1) % authors_tmp.count() == (authors_tmp.rownum - 1) % authors_tmp.count()
).select(
    books_tmp.book_id,
    authors_tmp.author_id
)

book_authors.show(5)
book_authors.printSchema()


/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+-------+---------+
|book_id|author_id|
+-------+---------+
|      1|       10|
|     10|     1000|
|    100|    10000|
|   1000|   100000|
|  10000|  1000007|
+-------+---------+
only showing top 5 rows
root
 |-- book_id: string (nullable = true)
 |-- author_id: string (nullable = true)



In [0]:
spark.catalog.clearCache()
spark.sql("CLEAR CACHE")


DataFrame[]

In [0]:
base = "abfss://lakehouse@goodreadsreviews60314097.dfs.core.windows.net"

books        = spark.read.parquet(f"{base}/processed/books/")
authors      = spark.read.parquet(f"{base}/processed/authors/")
reviews_clean = spark.read.parquet(f"{base}/processed/reviews/")  # or your cleaned path


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W

# Number of authors
n_auth = authors.count()
if n_auth == 0:
    raise Exception("`authors` is empty; cannot build bridge.")

# Row-number each DF deterministically
books_w = books.select("book_id").withColumn("rn_b", F.row_number().over(W.orderBy("book_id")))
authors_w = authors.select("author_id").withColumn("rn_a", F.row_number().over(W.orderBy("author_id")))

# Round-robin assign each book to an author
book_authors = (books_w
    .withColumn("join_key", ((F.col("rn_b") - 1) % F.lit(n_auth)) + 1)
    .join(authors_w.withColumn("join_key", F.col("rn_a")), on="join_key", how="inner")
    .select("book_id", "author_id")
    .dropDuplicates(["book_id", "author_id"])
)

book_authors.show(5)


/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+-------+---------+
|book_id|author_id|
+-------+---------+
|      1|       10|
|     10|     1000|
|    100|    10000|
|   1000|   100000|
|  10000|  1000007|
+-------+---------+
only showing top 5 rows


In [0]:
curated = (
    reviews_clean.alias("r")
    .join(books.select("book_id", "title").alias("b"), on="book_id", how="inner")
    .join(book_authors.alias("ba"), on="book_id", how="inner")
    .join(authors.select("author_id", "name").alias("a"), on="author_id", how="inner")
    .select(
        "r.review_id",
        "r.book_id",
        "b.title",
        "a.author_id",
        "a.name",
        "r.user_id",
        "r.rating",
        "r.review_text"
    )
)


/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
curated.printSchema()
curated.show(5, truncate=False)


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)



/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+--------------------------------+--------+----------------------------------------------------------------------------------------------------+---------+-------------------------+--------------------------------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# Switch to the legacy Hive metastore (works even when UC is enabled)
spark.sql("USE CATALOG hive_metastore")
spark.sql("USE SCHEMA default")

# Register your in-memory DataFrame as a managed table
curated.write.format("delta").mode("overwrite").saveAsTable("curated_reviews")

# Verify
spark.sql("SHOW TABLES IN hive_metastore.default").show()
spark.sql("SELECT COUNT(*) AS rows FROM hive_metastore.default.curated_reviews").show()
spark.sql("SELECT review_id, book_id, title, author_id, name, user_id, rating FROM hive_metastore.default.curated_reviews LIMIT 5").show(truncate=False)


/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
| default|curated_reviews|      false|
+--------+---------------+-----------+

+--------+
|    rows|
+--------+
|14971371|
+--------+

+--------------------------------+--------+----------------------------------------------------------------------------------------------------+---------+-------------------------+--------------------------------+------+
|review_id                       |book_id |title                                                                                               |author_id|name                     |user_id                         |rating|
+--------------------------------+--------+----------------------------------------------------------------------------------------------------+---------+-------------------------+--------------------------------+------+
|5c10ad6f049d5695c9d12355ba36ddd4|840918  |Spiritual Literacy: Reading the Sacred in 

In [0]:
# You're on hive_metastore.default already
spark.sql("SHOW TABLES IN hive_metastore.default").show()
spark.sql("SELECT COUNT(*) AS rows FROM hive_metastore.default.curated_reviews").show()
spark.sql("""
SELECT review_id, book_id, title, author_id, name, user_id, rating
FROM hive_metastore.default.curated_reviews
LIMIT 10
""").show(truncate=False)


+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
| default|curated_reviews|      false|
+--------+---------------+-----------+

+--------+
|    rows|
+--------+
|14971371|
+--------+

+--------------------------------+--------+----------------------------------------------------------------------------------------------------+---------+-------------------------+--------------------------------+------+
|review_id                       |book_id |title                                                                                               |author_id|name                     |user_id                         |rating|
+--------------------------------+--------+----------------------------------------------------------------------------------------------------+---------+-------------------------+--------------------------------+------+
|5c10ad6f049d5695c9d12355ba36ddd4|840918  |Spiritual Literacy: Reading the Sacred in 

In [0]:
# تحديد قاعدة البيانات
spark.sql("USE hive_metastore.default")

# عرض كل الجداول للتأكد أن الجدول موجود
print("📋 الجداول المسجلة حالياً:")
spark.sql("SHOW TABLES").show(truncate=False)

# عرض عدد الصفوف في الجدول
print("📊 عدد الصفوف في جدول curated_reviews:")
spark.sql("SELECT COUNT(*) AS total_rows FROM curated_reviews").show()

# عرض بعض الصفوف من الجدول للتأكد من وجود البيانات
print("🧾 بعض العينات من البيانات:")
spark.sql("""
SELECT review_id,
       book_id,
       title,
       author_id,
       name,
       user_id,
       rating,
       SUBSTRING(review_text, 1, 100) AS sample_review
FROM curated_reviews
LIMIT 10
""").show(truncate=False)

# عرض الإحصائيات الأساسية (مثل أقل وأعلى تقييم)
print("📈 إحصائيات بسيطة:")
spark.sql("""
SELECT
    MIN(rating) AS min_rating,
    MAX(rating) AS max_rating,
    AVG(rating) AS avg_rating
FROM curated_reviews
""").show()


📋 الجداول المسجلة حالياً:
+--------+---------------+-----------+
|database|tableName      |isTemporary|
+--------+---------------+-----------+
|default |curated_reviews|false      |
+--------+---------------+-----------+

📊 عدد الصفوف في جدول curated_reviews:
+----------+
|total_rows|
+----------+
|  14971371|
+----------+

🧾 بعض العينات من البيانات:
+--------------------------------+--------+----------------------------------------------------------------------------------------------------+---------+-------------------------+--------------------------------+------+-----------------------------------------------------------------------------------------------------+
|review_id                       |book_id |title                                                                                               |author_id|name                     |user_id                         |rating|sample_review                                                                                        |
+